In [1]:
pip install gymnasium

In [2]:
from time import sleep
import numpy as np
from IPython.display import clear_output
import gymnasium as gym
from gymnasium.envs.registration import register
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
import math

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
%cd /gdrive/MyDrive/SP 25/Reinforcement Learning/PPO

/gdrive/MyDrive/SP 25/Reinforcement Learning/PPO


In [10]:
from MiniPacManGym import MiniPacManEnv
register(
    id="MiniPacMan-v0",
    entry_point=MiniPacManEnv,
    max_episode_steps=20
)

env = gym.make("MiniPacMan-v0", render_mode="human", frozen_ghost=True)

/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment MiniPacMan-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [11]:
class CNN(nn.Module):
  def __init__(self, actions):
    super(CNN, self).__init__()

    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)

    self.fc1 = nn.Linear(6 * 6 * 64, 128)
    self.fc2 = nn.Linear(128, 4)

    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = x.view(x.size(0), -1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    x = self.softmax(x)

    return x

In [7]:
pi_network = CNN(4)
optimizer = optim.Adam(pi_network.parameters(), lr=0.001)

In [12]:
M = 1000
N = 10
gamma = 0.99

for e in range(M):
  B = []
  for i in range(N):
    done = False
    truncated = False
    state, info = env.reset()
    tau = []
    while not done and not truncated:
      state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

      action_probs = pi_network(state)
      action = torch.multinomial(action_probs, 1).item()

      next_state, reward, done, truncated, info = env.step(action)
      log_prob = torch.log(action_probs[0, action])
      tau.append((log_prob, reward))

      state = next_state

    rewards = [tau[t][1] for t in range(len(tau))]
    discounted_rewards = [gamma**t * rewards[t] for t in range(len(rewards))]
    R = sum(discounted_rewards)
    R = (R-np.mean(discounted_rewards)) / (np.std(discounted_rewards) + 1e-8)
    B.append(R * tau[-1][0])

  optimizer.zero_grad()
  loss = -torch.mean(torch.stack(B))
  loss.backward()
  optimizer.step()

  if e > 0 and e % 100 == 0:
        print(f'episode: {e}, loss: {loss.item()}')

episode: 100, loss: -0.0
episode: 200, loss: -0.0
episode: 300, loss: 7.1971344368648715e-06
episode: 400, loss: 3.1987215720619133e-07
episode: 500, loss: 4.264962285560614e-07
episode: 600, loss: 3.1987215720619133e-07
episode: 700, loss: 3.7318417867027165e-07
episode: 800, loss: 3.1987215720619133e-07
episode: 900, loss: 3.7318417867027165e-07


In [ ]:
obs, info = env.reset()
done = False
truncated = False

while not done and not truncated:
    env.render()
    obs=torch.tensor(obs,dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    action_probs = pi_network(obs)
    action = torch.multinomial(action_probs, 1).item()

    obs, reward, done, truncated, info = env.step(action)
    sleep(1)
    clear_output(wait=True)

env.render()
env.close()

xxxxxx
xᗧ···x
x··ᗣ·x
x····x
x···◯x
xxxxxx

